In [2]:
import numpy as np
import matplotlib.pyplot as plt
import os.path
import os
from scipy.interpolate import interp1d
from scipy.signal import butter,filtfilt
from scipy.optimize import curve_fit
from scipy import odr

# Imort frequency files

In [16]:
def import_data (filepath, number_of_headers, number_of_columns):
    
    unsorted_data = []
    f = open(filepath, 'r')
    
    for i in np.arange(number_of_headers):
        f.readline()
    
    if number_of_columns == 'all':
        for line in f:
            line = line.strip()
            line = line.split()
            for i in np.arange(len(line)):
                line[i] = float(line[i])
            unsorted_data.append(line)
    else:
        for line in f:
            line=line.strip()
            line=line.split()
            for i in np.arange(number_of_columns):
                line[i] = float(line[i])
            unsorted_data.append(line[:number_of_columns])
    
    unsorted_data.sort(key = lambda x:x[0])     # this sorts all data files according to temperature    
    data = np.array(unsorted_data).transpose()
    
    return (data)

# throw out data that is too far away (i.e. obviously wrong)

In [ ]:
def clean_data (T, f, gamma, epsilon):
    df = np.gradient(f)
    ddf = np.gradient(df)
    
    m = np.mean(abs(ddf))
    std = np.std(abs(ddf))
    
    mask = abs(abs(ddf)-m)<epsilon*std
    fnew = f[mask]
    Tnew = T[mask]
    gammanew = gamma[mask]
    
    return ( Tnew, fnew, gammanew )

# interpolate the data

In [ ]:
# this part of the code is to throw away all data points that have the same temperature
# this is necessary to being able to interpolate the data

def interpolate (T, f, gamma, Tinterpolation, kind):
    index = []
    for i in np.arange(len(T)-1):
        if T[i] == T[i+1]:
            index.append(i)
    
    maskint = np.ones(len(T))
    for k in index:
        maskint[k] = 0
    maskint = (maskint == 1)
    # the lines of code above throw away all data which has the same temperature values
    
    f_int = interp1d(T[maskint], f[maskint], kind=kind)(Tinterpolation)
    g_int = interp1d(T[maskint], gamma[maskint], kind=kind)(Tinterpolation)
    
    return (f_int, g_int)

# apply low-pass filter to data

In [ ]:
def butter_lowpass_filter_ready (f, cutoff_frequency):
    
    # Filter requirements.
    period = 5.0         # Sample Period
    fs = 50       # sample rate, Hz
    cutoff = cutoff_frequency      # desired cutoff frequency of the filter, Hz , slightly higher than actual 1.2 Hz
    
    nyq = 0.5 * fs  # Nyquist Frequency
    
    order = 2       # sin wave can be approx represented as quadratic
    n = int(period * fs) # total number of samples
    
    
    normal_cutoff = cutoff / nyq
    # Get the filter coefficients 
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, f)
    return y

# Save data

In [1]:
def save (data, text, filename):
    if os.path.isfile(filename) == True:
        x='w'
    else:
        x='x'
        
    with open(filename, x) as g:
        if text != 'None':
            for line in text:
                a = ''
                for i in line:
                    a = a + i + '\t'
                a = a[:-1] + '\n'
                g.write(a)
            
        for line in data:
            a = ''
            for i in line:
                a = a + str(i) + '\t'
            a = a[:-1] + '\n'
            g.write(a)

# calculate elastic constants from resonance frequencies

In [ ]:
def elastic_constants(resonance_frequencies, highT_frequencies, a, ChighT):
    dfof_ht = np.array([(resonance_frequencies[i]-highT_frequencies[i])/highT_frequencies[i] for i in np.arange(len(highT_frequencies))])
    
    step1 = np.linalg.inv(np.matmul(a.transpose(), a))
    step1[abs(step1)<1e-14] = 0
    
    step2 = np.matmul(step1, a.transpose())
    step2[abs(step2)<1e-14] = 0
    
    step3 = 2* np.array([np.matmul(step2, dfof_ht.transpose()[i]) for i in np.arange(len(dfof_ht.transpose()))])
    
    CofT = np.array([(step3.transpose()[i]*ChighT[i])+ChighT[i] for i in np.arange(len(ChighT))])
    
    return CofT